Instagram all posts images Downloader from UserHome
完成形
ユーザーの画像をすべてダウンロードする

In [ ]:
from time import sleep

import os
import requests
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())

def login():
    #login
    driver.get('https://www.instagram.com/accounts/login/')

    username = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
    password = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))
    username.clear()
    password.clear()
    username.send_keys("your user id")
    password.send_keys("your password")
    button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
    #save your login info?
    not_now = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]'))).click()

login()

In [ ]:
number_of_imgs = 0
number_of_videos = 0
number_of_posts = 0
src_list_image = []
src_list_video = []
video_urls = []
count_post = 0

keyword = 'user id'

def GoToProfile(id):
    driver.get(f'https://www.instagram.com/{id}/')
    sleep(2)
    return id

def GoToPost():

    print('Finding where the post is')

    last_height = 0
    posts_urls = []
    a_tag_list = []
    count = 0
    while True:
        #find 'a' tag
        article = driver.find_element(By.TAG_NAME, 'article')
        a_tag = article.find_element(By.TAG_NAME, 'a').click()

        #Check a_tag before scan attributes

        print('start to scan a_tag')
        
        return count, posts_urls

def ScanImages():
    driver.implicitly_wait(10)
    sleep(2)
    global count_post
    global number_of_imgs
    global number_of_videos
    global number_of_posts
    number_of_posts += 1
    sleep(2)
    while True:
        sleep(3)

        # if you want to click like, remove →"#"ClickLike()
        #ClickLike()
        article = driver.find_elements(By.TAG_NAME, 'article')
        img_tags = article[1].find_elements(By.CSS_SELECTOR, 'img[style="object-fit: cover;"]')
            
        for i in img_tags:
            print('load imgs')
            img_src = i.get_attribute('src')
            if not img_src in src_list_image:
                src_list_image.append(img_src)
                number_of_imgs += 1

        try:
            driver.find_element(By.CSS_SELECTOR, "button[aria-label='Next']").click()
            sleep(2)
            driver.find_element(By.CSS_SELECTOR, "button[aria-label='Next']").click()
            continue
        except Exception as e:
            break
    try:
        driver.find_element(By.CSS_SELECTOR, "svg[aria-label='Next']").click()
        count_post += 1
        print(f'Post {count_post}')
        ScanImages()
    except Exception as e:
        driver.find_element(By.CSS_SELECTOR, "svg[aria-label='Close']").click()
        print('All images and videos were scaned')

    return number_of_posts, number_of_imgs, number_of_videos

def ClickLike():
    article = driver.find_elements(By.TAG_NAME, 'article')
    section = article[1].find_element(By.TAG_NAME, 'section')
    clike_like = section.find_element(By.TAG_NAME, 'svg').get_attribute('aria-label')
    if clike_like == 'Like':
        section.find_element(By.TAG_NAME, 'button').click()
        sleep(1)
        print('\nclicked like\n')

def SaveImages(dir_path):
    print('start to save imgs')
    post_stored_count = 0
    saved_images = 0
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    for img_src in src_list_image:
        print('saved')
        img_data = requests.get(img_src).content
        sleep(1)
        post_stored_count += 1
        with open(os.path.join(dir_path, (f'post_{id}_{post_stored_count}.png')), 'wb') as handle:
            handle.write(img_data)
        saved_images += 1
    return saved_images

def result():
    global number_of_imgs
    global number_of_videos
    global number_of_posts
    print(f"{number_of_posts} posts were visited")
    print(f"{number_of_imgs} images were scanned")
    print(f"{number_of_videos} videos were scanned")
    print(f'saved images {saved_images}')
    GoToProfile(id)

  
id = GoToProfile(keyword)
GoToPost()
number_of_posts, number_of_imgs, number_of_videos = ScanImages()
saved_images = SaveImages(f'InstagramDownloadFiles\{id}\Images')
result()